# Preprocess

In [1]:
# Import necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#reading the csv file 
file = 'data.csv'
df = pd.read_csv(file)

In [3]:
df.head()

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1


In [4]:
#%pip install nltk

In [5]:
# Install and import NLTK library
import nltk
nltk.__version__

'3.8.1'

In [6]:
df.shape

(20001, 2)

In [7]:
# Check for missing values
df=df[pd.to_numeric(df['tagging'], errors='coerce').notnull()]

In [8]:
df.shape

(20001, 2)

In [9]:
# Drop missing values
df = df.dropna(axis=0)

In [10]:
df.shape

(20001, 2)

In [11]:
import re
import nltk
import nltk.tokenize


In [12]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [13]:
# install sklearn
# %pip install -U scikit-learn

In [14]:
def preprocess_text(text):
    # Remove URLs, mentions, hashtags, and special characters
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r"@\w+", '', text)
    text = re.sub(r"#", '', text)
    text = re.sub(r"[^\w\s]", '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Stopword removal
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return ' '.join(tokens)

In [15]:
# Apply preprocessing to the "comments" column
df['cleaned_comments'] = df['comments'].map(preprocess_text)

In [16]:
df.head()

,comments,tagging,cleaned_comments
0,Get fucking real dude.,1,get fucking real dude
1,She is as dirty as they come and that crook ...,1,dirty come crook rengel dems fucking corrupt j...
2,why did you fuck it up. I could do it all day...,1,fuck could day let hour ping later sched writi...
3,Dude they dont finish enclosing the fucking s...,1,dude dont finish enclosing fucking shower hate...
4,WTF are you talking about Men? No men thats n...,1,wtf talking men men thats menage thats gay


In [17]:
# Delete the "comments" column
df = df.drop(columns=['comments'])

df.head()


,tagging,cleaned_comments
0,1,get fucking real dude
1,1,dirty come crook rengel dems fucking corrupt j...
2,1,fuck could day let hour ping later sched writi...
3,1,dude dont finish enclosing fucking shower hate...
4,1,wtf talking men men thats menage thats gay


In [18]:
# Save as a new CSV file
df.to_csv('cleaned_data.csv', index=False)

In [19]:
# check missing values
df.isnull().sum()

tagging             0
cleaned_comments    0
dtype: int64

In [20]:
# Check if there are empty strings in the 'cleaned_comments' column
empty_strings_count = df['cleaned_comments'].apply(lambda x: len(x.strip()) == 0).sum()
print(f"Number of empty strings: {empty_strings_count}")


Number of empty strings: 115


In [21]:
df['cleaned_comments'] = df['cleaned_comments'].apply(lambda x: x.strip() if len(x.strip()) > 0 else 'empty')

In [22]:
# Check if there are empty strings in the 'cleaned_comments' column
empty_strings_count = df['cleaned_comments'].apply(lambda x: len(x.strip()) == 0).sum()
print(f"Number of empty strings: {empty_strings_count}")


Number of empty strings: 0


In [23]:
df['cleaned_comments'] = df['cleaned_comments'].astype(str)


In [24]:
# Save as a new CSV file
df.to_csv('cleaned_data.csv', index=False)

# Training Testing

In [35]:
# 1. Import necessary libraries and read the cleaned data
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
import pandas as pd
import numpy as np
import pickle

# Read the cleaned data
file = 'cleaned_data.csv'

In [36]:
# 2. Splitting dataset for training and testing
# Split 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_comments'], 
    df['tagging'], 
    random_state=42, 
    test_size=0.20
)

# Explanation:
# - `train_test_split`: Divides the data into training and testing sets.
# - `random_state`: Ensures reproducibility of the split.
# - `test_size`: Allocates 20% of data for testing.

In [37]:
# 3. Vectorizing text data
# Using CountVectorizer to convert text into numerical features
count_vector = CountVectorizer()
X_train_vec = count_vector.fit_transform(X_train)  # Fit on training data
X_test_vec = count_vector.transform(X_test)       # Transform test data

# Explanation:
# - `CountVectorizer`: Converts text data into a matrix of token counts.
# - `fit_transform`: Learns the vocabulary and applies it to the training data.
# - `transform`: Converts test data into the same vocabulary space.


In [38]:
# 4. Define machine learning models and hyperparameters
# Dictionary of models with their respective hyperparameter grids
model_params = {
    'LinearSVC': {
        'model': LinearSVC(max_iter=10000, random_state=42),
        'params': {
            'C': [0.1, 1, 5, 10, 20],
        }
    },
    'MultinomialNB': {
        'model': MultinomialNB(),
        'params': {
            'alpha': np.linspace(0.5, 1.5, 6),
            'fit_prior': [True, False],
        }
    },
    'logistic_regression': {
        'model': LogisticRegression(random_state=42, multi_class='ovr', max_iter=10000),
        'params': {
            'C': [1, 5, 10],
            'solver': ['newton-cg', 'lbfgs', 'liblinear'],
        }
    },
    'KNeighborsClassifier': {
        'model': KNeighborsClassifier(p=2),
        'params': {
            'n_neighbors': [5, 9, 11, 23],
            'weights': ['uniform', 'distance'],
            'metric': ['euclidean', 'manhattan', 'minkowski'],
        }
    }
}

# Explanation:
# - Each model is paired with its hyperparameters for tuning.
# - GridSearchCV will explore combinations of these parameters.

In [39]:
# 5. Grid Search to find the best model and hyperparameters, then save models as .pkl
for model_name, mp in model_params.items():
    print(f"Training and saving {model_name}...")

    # Create and fit the vectorizer
    vectorizer = CountVectorizer()
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    # Initialize GridSearchCV with cross-validation
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(X_train_vec, y_train)  # Train the model with training data

    # Get the best model
    best_model = clf.best_estimator_

    # Save the model as a .pkl file
    with open(f'{model_name}.pkl', 'wb') as model_file:
        pickle.dump(best_model, model_file)

    # Save the vectorizer as a .pkl file
    with open(f'{model_name}_vectorizer.pkl', 'wb') as vectorizer_file:
        pickle.dump(vectorizer, vectorizer_file)

    # Optionally, print out the results for this model
    print(f"Model: {model_name}")
    print(f"Best Score: {clf.best_score_}")
    print(f"Best Params: {clf.best_params_}")
    print("-" * 40)

# Explanation:
# - `GridSearchCV`: Searches for the best combination of hyperparameters using cross-validation.
# - `cv=5`: Splits the training data into 5 folds for validation.
# - `clf.best_score_`: Best score achieved for the model.
# - `clf.best_params_`: Best hyperparameter combination.
# - Each model is saved as a `.pkl` file for later use.


Training and saving LinearSVC...


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\svm\_clas

Model: LinearSVC
Best Score: 0.8318749999999999
Best Params: {'C': 1}
----------------------------------------
Training and saving MultinomialNB...
Model: MultinomialNB
Best Score: 0.7675625
Best Params: {'alpha': 0.5, 'fit_prior': True}
----------------------------------------
Training and saving logistic_regression...
Model: logistic_regression
Best Score: 0.835625
Best Params: {'C': 10, 'solver': 'lbfgs'}
----------------------------------------
Training and saving KNeighborsClassifier...
Model: KNeighborsClassifier
Best Score: 0.7748125
Best Params: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}
----------------------------------------


In [40]:
# 6. Load the saved model and evaluate on the test set
metrics = []
for model_name in model_params.keys():
    print(f"Evaluating {model_name}...")

    # Load the model
    with open(f'{model_name}.pkl', 'rb') as model_file:
        model = pickle.load(model_file)

    # Load the vectorizer
    with open(f'{model_name}_vectorizer.pkl', 'rb') as vectorizer_file:
        vectorizer = pickle.load(vectorizer_file)

    # Transform the test data using the loaded vectorizer
    X_test_vec = vectorizer.transform(X_test)

    # Predict on the test set
    y_pred = model.predict(X_test_vec)

    # Compute evaluation metrics
    metrics.append({
        'model': model_name,
        'accuracy': accuracy_score(y_test, y_pred),
        'f1_score': f1_score(y_test, y_pred, average='weighted'),
        'precision': precision_score(y_test, y_pred, average='weighted'),
        'recall': recall_score(y_test, y_pred, average='weighted'),
        'confusion_matrix': confusion_matrix(y_test, y_pred).tolist()
    })

Evaluating LinearSVC...
Evaluating MultinomialNB...
Evaluating logistic_regression...
Evaluating KNeighborsClassifier...


In [41]:
# Convert metrics into a DataFrame for easier comparison
df_metrics = pd.DataFrame(metrics, columns=['model', 'accuracy', 'f1_score', 'precision', 'recall', 'confusion_matrix'])
print(df_metrics)

# Explanation:
# - After saving the models, they are loaded back for evaluation.
# - Metrics such as accuracy, F1 score, precision, and recall help compare the performance of models.
# - `confusion_matrix`: Provides insight into the distribution of predictions.
# - Each model is evaluated on the test set to determine its generalization performance.

                  model  accuracy  f1_score  precision    recall  \
0             LinearSVC  0.853537  0.855053   0.865780  0.853537   
1         MultinomialNB  0.798800  0.799645   0.801219  0.798800   
2   logistic_regression  0.852787  0.854253   0.863545  0.852787   
3  KNeighborsClassifier  0.782554  0.783543   0.841086  0.782554   

             confusion_matrix  
0  [[1984, 440], [146, 1431]]  
1  [[1979, 445], [360, 1217]]  
2  [[1994, 430], [159, 1418]]  
3   [[1614, 810], [60, 1517]]  


# Model Comparison

In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Best parameters for each model from GridSearchCV
best_params = {
    'LinearSVC': {'C': 1},
    'MultinomialNB': {'alpha': 0.5, 'fit_prior': True},
    'logistic_regression': {'C': 10, 'solver': 'liblinear'},
    'KNeighborsClassifier': {'metric': 'euclidean', 'n_neighbors': 23, 'weights': 'distance'}
}

In [38]:
# Models with best parameters
models = {
    'LinearSVC': LinearSVC(C=best_params['LinearSVC']['C'], max_iter=10000, random_state=42),
    'MultinomialNB': MultinomialNB(alpha=best_params['MultinomialNB']['alpha'], fit_prior=best_params['MultinomialNB']['fit_prior']),
    'logistic_regression': LogisticRegression(C=best_params['logistic_regression']['C'], 
                                              solver=best_params['logistic_regression']['solver'], 
                                              max_iter=10000, random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier(metric=best_params['KNeighborsClassifier']['metric'], 
                                                 n_neighbors=best_params['KNeighborsClassifier']['n_neighbors'], 
                                                 weights=best_params['KNeighborsClassifier']['weights'])
}

In [39]:
# List to store results
results = []

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on test data
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Append results to the list
    results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

ValueError: could not convert string to float: 'im fucking pissed shit'

In [ ]:
# Create a DataFrame for easy comparison
results_df = pd.DataFrame(results)

In [ ]:
# Display the comparison table
print("\nModel Performance Comparison:")
print(results_df.sort_values(by='F1 Score', ascending=False))